In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from importlib import reload
import datetime
from glob import glob
import json
import dask.dataframe as dd

from mfishtoolspy import mfishtoolspy as mft



In [2]:
# options for cluster grouping
ops = {
    # 'panel_name': 'Pan-neuronal', # GABAergic or Glutamatergic or Pan-neuronal
    'panel_name': 'GABAergic', # GABAergic or Glutamatergic or Pan-neuronal
    # 'full_panel_size': 28,
    'full_panel_size': 22,
    # 'starting_genes': ["Gad2","Slc17a7","Pvalb","Sst","Vip","Cck","Tac1","Npy","Crh","Necab1","Ptprt","Kirrel3","Penk","Hpse","Calb2","Chodl"],
    'starting_genes': ["Gad2","Slc17a7","Pvalb","Sst","Vip", "Sncg", "Lamp5", "Npy", "Calb2", "Tac1", "Cck", "Ndnf"],
    'layer_1234_filter': True,
    'GABAergic_group_level': 'cluster', # class, subclass, supertype, or cluster
    'GABAergic_mapping_level': 'cluster',
    'Glutamatergic_group_level': 'subclass', # class, subclass, supertype, or cluster
    'Glutamatergic_mapping_level': 'subclass',
    'GABAergic_other_group_level': 'subclass', # class, subclass, supertype, or cluster
    'GABAergic_other_mapping_level': 'subclass',
    'Glutamatergic_other_group_level': 'class', # class, subclass, supertype, or cluster
    'Glutamatergic_other_mapping_level': 'class',
    'blend_supertypes': False,  # Don't know if I'm going to keep this
    'remove_redundant_genes': False, # from the starting_genes list
    'remove_redundant_genes_threshold': 0.95, # threshold for removing redundant genes from normalized accuracy
    'L1234_layer_threshold': 0.15,
    'L6_layer_threshold': 0.7,
    'L1234_labels': ['L1', 'L1-L2/3', 'L1-L4', 'L2/3', 'L2/3-L4', 'L4'],
    'L6_labels': ['L6', 'L6b'],
    'max_on': 5000,
    'max_off': 1129,
    'min_on': 10,
    'max_fraction_on_clusters': 0.5,
    'num_subsample': 50,
}

# paths to the data
data_folder = Path(r'\\allen\programs\mindscope\workgroups\learning\jinho\gene_panel_selection\data\mouse_VISp_gene_expression_matrices_2018-06-14')
output_folder = Path(r'\\allen\programs\mindscope\workgroups\learning\jinho\gene_panel_selection\results')

In [3]:
# preprocess options
if 'GABAergic' in ops['panel_name']:
    ops['keep_class'] = ['GABAergic']
elif 'Glutamatergic' in ops['panel_name']:
    ops['keep_class'] = ['Glutamatergic']
elif 'Pan-neuronal' in ops['panel_name']:
    ops['keep_class'] = ['GABAergic', 'Glutamatergic']
else:
    raise ValueError('panel_name must be GABAergic, Glutamatergic, or Pan-neuronal')

level_hierarchy = {'class': 0, 'subclass': 1, 'supertype': 2, 'cluster': 3}
assert level_hierarchy[ops['GABAergic_group_level']] <= level_hierarchy[ops['GABAergic_mapping_level']]
assert level_hierarchy[ops['GABAergic_other_group_level']] <= level_hierarchy[ops['GABAergic_other_mapping_level']]
assert level_hierarchy[ops['Glutamatergic_group_level']] <= level_hierarchy[ops['Glutamatergic_mapping_level']]
assert level_hierarchy[ops['Glutamatergic_other_group_level']] <= level_hierarchy[ops['Glutamatergic_other_mapping_level']]


In [4]:
# load and pre-process data (takes about 1 min)

# read annotation
annotation = pd.read_feather(data_folder / 'anno.feather')
# read data
# TODO: check where this data is coming from. Values are similar to cpm but not exactly the same
cpm_data = pd.read_feather(data_folder / 'exon_cpm.feather')
tpm_data = pd.read_feather(data_folder / 'exon_tpm.feather')

annotation.set_index('sample_id', inplace=True, drop=True)
# data.set_index('gene', inplace=True, drop=True) # only necessary with data_t.feather

# preprocessing
# Removing 'X" in column? from Hannah's code. Don't know when this happens, but leave them here just in case.
if 'X' in cpm_data.columns:
    print('Dropping "X" column from data')
    cpm_data = cpm_data.drop(columns=['X'])
if 'X' in tpm_data.columns:
    print('Dropping "X" column from data')
    tpm_data = tpm_data.drop(columns=['X']) 
if 'X' in annotation.columns:
    print('Dropping "X" column fro annotation')
    annotation = annotation.drop(columns=['X'])

# change the row order of annotation to match the order of columns in data
# annotation = annotation.loc[data.columns]  # don't need this, but add assert statement to check
assert np.all(annotation.index.values == tpm_data.columns.values)
assert np.all(annotation.index.values == cpm_data.columns.values)

# data conversion to log2
tpm_log2 = np.log2(tpm_data + 1)
cpm_log2 = np.log2(cpm_data + 1)

# read supertype information
# TODO: re-define supertype (will be addressed in another notebook)
supertype_folder = Path('//allen/programs/mindscope/workgroups/omfish/hannahs/mfish_project/gene_panels')
supertype_fn = supertype_folder / 'tasic2018_supertypes_manual_v2.xlsx'
sheet_name = 'all_supertypes_v2'
supertype = pd.read_excel(supertype_fn, sheet_name=sheet_name)
supertype.rename(columns={'Cell Type': 'cell_type', 'Supertype': 'supertype'}, inplace=True)
supertype.cell_type = supertype.cell_type.str.replace('\xa0', ' ')
supertype.supertype = supertype.supertype.str.replace('\xa0', ' ')
assert np.all([ct in annotation['cluster_label'].values for ct in supertype.cell_type.values])
supertype.set_index('cell_type', inplace=True, drop=True)

annotation['supertype_label'] = annotation.cluster_label.map(supertype.supertype)

C:\Users\jinho.kim\AppData\Local\Temp\ipykernel_42460\2181021420.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  annotation['supertype_label'] = annotation.cluster_label.map(supertype.supertype)


In [5]:
# assign mappings and groups and update the ops dictionary

group_label = f'{ops["Glutamatergic_group_level"]}_label'
mapping_label = f'{ops["Glutamatergic_mapping_level"]}_label'
temp_annotation = annotation.query('class_label=="Glutamatergic"')

# filtering and assigning group_label
# Group means target clustering that I want to classify eventually
# There is also mapping_label, to which I want to match first before calculating classification accuracy
# E.g., match individual samples to cluster level but then classify them at supertype level
# E.g., classification can happen in mixed level, some at the cluster level while others in subclass level

# If "other" group and mapping levels are different from group and mapping levels
# then assign relevant labels to "other" group and mapping levels
#   "other" is relevant only when filtering by the layers
#   It means the rest of groups of mappings that can be within the imaging regime but we want to exclude from further analysis

# Filtering based on layer abundance
# TODO: test thresholds. 
# TODO: Better to do this from merFISH data

keep_groups = []
keep_mappings = []
other_groups = []
other_mappings = []

if 'Glutamatergic' in ops['keep_class']:
    # Assign group and mapping labels
    group_level_label = f'{ops["Glutamatergic_group_level"]}_label'
    group_label_list = [f'Glutamatergic {gl}' for gl in annotation.loc[annotation.class_label == 'Glutamatergic', group_level_label]] # To disambiguate from GABAergic groups
    annotation.loc[annotation.class_label == 'Glutamatergic', 'group_label'] = group_label_list
    mapping_level_label = f'{ops["Glutamatergic_mapping_level"]}_label'
    mapping_label_list = [f'Glutamatergic {ml}' for ml in annotation.loc[annotation.class_label == 'Glutamatergic', mapping_label]] # To disambiguate from GABAergic mappings
    annotation.loc[annotation.class_label == 'Glutamatergic', 'mapping_label'] = mapping_label_list
    
    # Assign keep and other groups.
    # If lower than class level, then consider layer filtering (L1234 only for now)
    if ops['Glutamatergic_group_level'] == 'class':
        keep_groups += ['Glutamatergic']
    else:
        temp_annotation = annotation.query('class_label=="Glutamatergic"')
        if ops['layer_1234_filter']:
            keep_groups += [gl for gl in temp_annotation['group_label'].unique().tolist() if gl.split(' ')[1] in ['L2/3', 'L4']] # Have class label at the beginning
            # Process "other" groups
            other_group_level_label = f'{ops["Glutamatergic_other_group_level"]}_label'
            temp_other_groups = [gl for gl in temp_annotation['group_label'].unique().tolist() if gl.split(' ')[1] in ['L5', 'NP']] # Have class label at the beginning
            if other_group_level_label == group_level_label: # no need to change group labels
                other_groups += temp_other_groups
            else:
                # Add L5 after class label
                other_group_labels = [f'Glutamatergic L5 {gl}' for gl in annotation[annotation['group_label'].isin(temp_other_groups)][other_group_level_label].values]
                annotation.loc[annotation['group_label'].isin(temp_other_groups), 'group_label'] = other_group_labels
                other_groups += np.unique(other_group_labels).tolist()
        else:
            keep_groups += temp_annotation['group_label'].unique().tolist()
    # Assign keep and other mappings.
    if ops['Glutamatergic_mapping_level'] == 'class':
        keep_mappings += ['Glutamatergic']
    else:
        temp_annotation = annotation.query('class_label=="Glutamatergic"')
        if ops['layer_1234_filter']:
            keep_mappings += [ml for ml in temp_annotation['mapping_label'].unique().tolist() if ml.split(' ')[1] in ['L2/3', 'L4']] # Have class label at the beginning
            # Process "other" groups
            other_mapping_level_label = f'{ops["Glutamatergic_other_mapping_level"]}_label'
            temp_other_mappings = [ml for ml in temp_annotation['mapping_label'].unique().tolist() if ml.split(' ')[1] in ['L5', 'NP']] # Have class label at the beginning
            if other_mapping_level_label == mapping_level_label: # no need to change mapping labels
                other_mappings += temp_other_mappings
            else:
                # Add L5 after class label
                other_mapping_labels = [f'Glutamatergic L5 {ml}' for ml in annotation[annotation['mapping_label'].isin(temp_other_mappings)][other_mapping_level_label].values]
                annotation.loc[annotation['mapping_label'].isin(temp_other_mappings), 'mapping_label'] = other_mapping_labels
                other_mappings += np.unique(other_mapping_labels).tolist()
        else:
            keep_mappings += temp_annotation['mapping_label'].unique().tolist()

# Same for GABAergic
# Except for filtering, now we are using scRNAseq layer-enriched data with thresholds
if 'GABAergic' in ops['keep_class']:
    # Assign group and mapping labels
    group_level_label = f'{ops["GABAergic_group_level"]}_label'
    group_label_list = [f'GABAergic {gl}' for gl in annotation.loc[annotation.class_label == 'GABAergic', group_level_label]] # To disambiguate from Glutamatergic groups
    annotation.loc[annotation.class_label == 'GABAergic', 'group_label'] = group_label_list
    mapping_level_label = f'{ops["GABAergic_mapping_level"]}_label'
    mapping_label_list = [f'GABAergic {ml}' for ml in annotation.loc[annotation.class_label == 'GABAergic', mapping_level_label]] # To disambiguate from Glutamatergic mappings
    annotation.loc[annotation.class_label == 'GABAergic', 'mapping_label'] = mapping_label_list
    
    # Assign keep and other groups.
    # If lower than class level, then consider layer filtering (L1234 only for now)
    # Also need to name them different (adding L5 in front of the group and mapping labels)
    if ops['GABAergic_group_level'] == 'class':
        keep_groups += ['GABAergic']
    else:
        temp_annotation = annotation.query('class_label=="GABAergic"')
        if ops['layer_1234_filter']:
            # Filtering process based on the layer abundance
            layer_df = annotation.query('class_label=="GABAergic"')[['layer_label', 'cluster_label']].copy()
            layer_table = layer_df.groupby(['layer_label', 'cluster_label']).size().unstack(fill_value=0)
            prop_table = layer_table.div(layer_table.sum(axis=0), axis=1)
            L1234_prop_sum = prop_table.loc[ops['L1234_labels']].sum(axis=0)
            L1234_inh_types = set(L1234_prop_sum[L1234_prop_sum >= ops['L1234_layer_threshold']].index.values)
            not_L1234_inh_types = set(layer_df.cluster_label).difference(L1234_inh_types)
            L6_prop_sum = prop_table.loc[ops['L6_labels']].sum(axis=0)
            L6_inh_types = set(L6_prop_sum[L6_prop_sum >= ops['L6_layer_threshold']].index.values)
            L5_inh_types = not_L1234_inh_types.difference(L6_inh_types)
            # L1234_inh_types are going to be kept
            # L5_inh_types are going to be "other"
            # Ignore L6_inh_types (assume they won't be imaged)
            keep_annotation = temp_annotation[temp_annotation['cluster_label'].isin(L1234_inh_types)]
            other_annotation = temp_annotation[temp_annotation['cluster_label'].isin(L5_inh_types)]
            
            keep_groups += keep_annotation['group_label'].unique().tolist()

            # Process "other" groups
            other_group_level_label = f'{ops["GABAergic_other_group_level"]}_label'
            temp_other_groups = other_annotation['group_label'].unique().tolist()
            if other_group_level_label == group_level_label: # no need to change group labels
                other_groups += temp_other_groups
            else:
                # Add L5 after class label
                other_group_labels = [f'GABAergic L5 {gl}' for gl in annotation[annotation['group_label'].isin(temp_other_groups)][other_group_level_label].values]
                annotation.loc[annotation['group_label'].isin(temp_other_groups), 'group_label'] = other_group_labels
                other_groups += np.unique(other_group_labels).tolist()
        else:
            keep_groups += temp_annotation['group_label'].unique().tolist()
    # Assign keep and other mappings.
    if ops['GABAergic_mapping_level'] == 'class':
        keep_mappings += ['GABAergic']
    else:
        temp_annotation = annotation.query('class_label=="GABAergic"')
        if ops['layer_1234_filter']:
            # Filtering process should have been done already in the above if clause
            # keep_annotation and other_annotation are already defined
            keep_mappings += keep_annotation['mapping_label'].unique().tolist()
            
            # Process "other" groups
            other_mapping_level_label = f'{ops["GABAergic_other_mapping_level"]}_label'
            temp_other_mappings = other_annotation['mapping_label'].unique().tolist()
            if other_mapping_level_label == mapping_level_label: # no need to change mapping labels
                other_mappings += temp_other_mappings
            else:
                # Add L5 after class label
                other_mapping_labels = [f'GABAergic L5 {ml}' for ml in annotation[annotation['mapping_label'].isin(temp_other_mappings)][other_mapping_level_label].values]
                annotation.loc[annotation['mapping_label'].isin(temp_other_mappings), 'mapping_label'] = other_mapping_labels
                other_mappings += np.unique(other_mapping_labels).tolist()
        else:
            keep_mappings += temp_annotation['mapping_label'].unique().tolist()

# assign cluster to nan mapping labels
# for filtering using "off clusters" information (e.g., glial cells)
annotation.loc[annotation['mapping_label']=='nan', 'mapping_label'] = annotation.loc[annotation['mapping_label']=='nan', 'cluster_label'] 


ops['keep_mappings'] = keep_mappings
ops['other_mappings'] = other_mappings
ops['keep_groups'] = keep_groups
ops['other_groups'] = other_groups

C:\Users\jinho.kim\AppData\Local\Temp\ipykernel_42460\2259671364.py:82: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  annotation.loc[annotation.class_label == 'GABAergic', 'group_label'] = group_label_list
C:\Users\jinho.kim\AppData\Local\Temp\ipykernel_42460\2259671364.py:85: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  annotation.loc[annotation.class_label == 'GABAergic', 'mapping_label'] = mapping_label_list


In [6]:
# calculate proportions and medians for mapping labels and for group labels
# Takes about 1 min
expr_thresh = 1
# make data_log2 to have another level of columns with matching cluster names per cell ID
tpm_log2_cluster = tpm_log2.copy().T
gene_column_names = list(tpm_log2_cluster.columns)
assert np.all(tpm_log2.columns == annotation.index.values)
# groupby cluster and calculate median and proportion
tpm_log2_cluster['mapping_label'] = annotation['mapping_label']
tpm_log2_cluster['group_label'] = annotation['group_label']
tpm_median_per_mapping = tpm_log2_cluster[gene_column_names + ['mapping_label']].groupby('mapping_label').median().T
tpm_prop_expr_mapping = tpm_log2_cluster[gene_column_names + ['mapping_label']].groupby('mapping_label').apply(lambda x: (x > expr_thresh).mean(axis=0)).T
tpm_median_per_group = tpm_log2_cluster[gene_column_names + ['group_label']].groupby('group_label').median().T
tpm_prop_expr_group = tpm_log2_cluster[gene_column_names + ['group_label']].groupby('group_label').apply(lambda x: (x > expr_thresh).mean(axis=0)).T
assert np.all(tpm_prop_expr_mapping.index.values == tpm_median_per_mapping.index.values)
assert np.all(tpm_prop_expr_group.index.values == tpm_median_per_group.index.values)
assert np.all(tpm_prop_expr_mapping.index.values == tpm_log2.index.values)
assert np.all(tpm_prop_expr_group.index.values == tpm_log2.index.values)

## Filter

In [7]:
tpm_summary_data = 2**tpm_median_per_mapping - 1
run_genes, filtered_out_genes = \
    mft.filter_panel_genes(tpm_summary_data,
                            prop_expr=tpm_prop_expr_mapping,
                            on_clusters=keep_mappings + other_mappings,
                            starting_genes=ops['starting_genes'],
                            off_clusters=list(annotation.query('class_label=="Non-Neuronal"').mapping_label.unique()),
                            max_on=ops['max_on'],
                            max_off=ops['max_off'])

1806 total genes pass constraints prior to binary score calculation.


## Build

In [8]:
reload(mft)

<module 'mfishtoolspy.mfishtoolspy' from 'c:\\users\\jinho.kim\\github\\mfishtoolspy\\mfishtoolspy\\mfishtoolspy.py'>

In [16]:
reload(mft)

<module 'mfishtoolspy.mfishtoolspy' from 'c:\\users\\jinho.kim\\github\\mfishtoolspy\\mfishtoolspy\\mfishtoolspy.py'>

In [9]:
run_mappings = keep_mappings + other_mappings
keep_samples = annotation[annotation['mapping_label'].isin(run_mappings)].index.values


built_panel, metric = mft.build_mapping_based_marker_panel(
    map_data=tpm_log2.loc[run_genes, keep_samples],
    mapping_median_data=tpm_median_per_mapping.loc[run_genes, run_mappings],
    mapping_call=annotation.loc[keep_samples, 'mapping_label'],
    mapping_to_group=annotation.loc[keep_samples, ['mapping_label','group_label']].drop_duplicates().set_index('mapping_label').loc[run_mappings]['group_label'],
    group_median_data=tpm_median_per_group.loc[run_genes, keep_groups + other_groups],
    panel_size=ops['full_panel_size'],
    num_subsample=ops['num_subsample'],
    current_panel=ops['starting_genes'].copy(),
)

c:\Users\jinho.kim\Anaconda3\envs\mfish\lib\site-packages\distributed\client.py:3362: UserWarning: Sending large graph of size 24.31 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


Added Parm1 with average cluster distance 0.193 [12].


c:\Users\jinho.kim\Anaconda3\envs\mfish\lib\site-packages\distributed\client.py:3362: UserWarning: Sending large graph of size 24.31 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


Added Npy2r with average cluster distance 0.169 [13].


c:\Users\jinho.kim\Anaconda3\envs\mfish\lib\site-packages\distributed\client.py:3362: UserWarning: Sending large graph of size 24.31 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


...

Added Reln with average cluster distance 0.109 [19].


c:\Users\jinho.kim\Anaconda3\envs\mfish\lib\site-packages\distributed\client.py:3362: UserWarning: Sending large graph of size 24.31 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


Added Lypd1 with average cluster distance 0.104 [20].


c:\Users\jinho.kim\Anaconda3\envs\mfish\lib\site-packages\distributed\client.py:3362: UserWarning: Sending large graph of size 24.31 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


Added Rgs12 with average cluster distance 0.100 [21].


In [ ]:
run_mappings = keep_mappings + other_mappings
keep_samples = annotation[annotation['mapping_label'].isin(run_mappings)].index.values

built_panel, metric = mft.build_mapping_based_marker_panel(
    map_data=tpm_log2.loc[run_genes, keep_samples],
    mapping_median_data=tpm_median_per_mapping.loc[run_genes, run_mappings],
    mapping_call=annotation.loc[keep_samples, 'mapping_label'],
    mapping_to_group=annotation.loc[keep_samples, ['mapping_label','group_label']].drop_duplicates().set_index('mapping_label').loc[run_mappings]['group_label'],
    group_median_data=tpm_median_per_group.loc[run_genes, keep_groups + other_groups],
    panel_size=ops['full_panel_size'],
    num_subsample=ops['num_subsample'],
    current_panel=ops['starting_genes'].copy(),
    use_parallel=False,
    num_iter_each_addition=1,
)

## Save the panel and option

In [19]:
now = datetime.datetime.now()
date = now.strftime("%Y_%m_%d")

save_ops_fn_base = f'ops_{date}_*.json'
save_results_fn_base = f'results_{date}_*.json'

prev_ops_fn_list = glob(str(output_folder / save_ops_fn_base))
prev_results_fn_list = glob(str(output_folder / save_results_fn_base))

if (len(prev_ops_fn_list) == 0) and (len(prev_results_fn_list) == 0):
    max_prev_num = -1
else:
    max_prev_num = max([int(fn.split('.')[0].split('_')[-1]) for fn in prev_ops_fn_list] +
                        [int(fn.split('.')[0].split('_')[-1]) for fn in prev_results_fn_list])
curr_num = max_prev_num + 1

save_ops_fn = output_folder / f'ops_{date}_{curr_num:02}.json'
save_results_fn = output_folder / f'results_{date}_{curr_num:02}.json'

results = {'run_genes': run_genes,
           'built_panel': built_panel,
           'metric': metric}


with open(save_ops_fn, 'w') as f:
    json.dump(ops, f)
with open(save_results_fn, 'w') as f:
    json.dump(results, f)